[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# DIY Covid-19 Dashboard

In [1]:
# setup
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100


In [2]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_data(data):
    datalist=data['data']

    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()

    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])

    index=pd.date_range(startdate, enddate, freq='D')
    timeseriesdf=pd.DataFrame(index=index, columns=['hospital', 'deaths'])

    for entry in datalist:
        date=parse_date(entry['date'])
        for column in ['hospital', 'deaths']:
            if pd.isna(timeseriesdf.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                timeseriesdf.loc[date, column]=value
            
    timeseriesdf.fillna(0.0, inplace=True)

    return timeseriesdf


In [3]:
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """

    print("Getting data...")

    filters = [
        'areaType=overview',
    ]

    # values here are the names of the PHE metrics
    structure = {
        "date": "date",
        "hospital": "newAdmissions",
        "deaths": "cumDeaths28DaysByDeathDateRate"    
    }

    api = Cov19API(filters=filters, structure=structure)

    timeseries=api.get_json()

    with open("timeseries.json", "wt") as OUTF:
        json.dump(timeseries, OUTF)
    print("... data retrieved")
    return timeseries

In [4]:
try:
    with open("timeseries.json", "rt") as INFILE:
        data=json.load(INFILE)
except:
    data = access_api()

timeseriesdf = wrangle_data(data)
timeseriesdf.to_pickle("timeseriesdf.pkl")
timeseriesdf=pd.read_pickle("timeseriesdf.pkl")

Getting data...
... data retrieved


In [5]:
series=wdg.SelectMultiple(
    options=['hospital', 'deaths'],
    value=['hospital', 'deaths'],
    rows=3,
    description='Stats:',
    disabled=False
)

controls=wdg.HBox([series])

def timeseries_graph(gcols):
    ncols=len(gcols)
    if ncols>0:
        timeseriesdf[list(gcols)].plot()
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); capture output in variable graph   
graph=wdg.interactive_output(timeseries_graph, {'gcols': series})

display(controls, graph)

Output()

In [6]:
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    apidata=access_api()
    global timeseriesdf
    timeseriesdf=wrangle_data(apidata)
    refresh_graph()
    apibutton.icon="check"
    
apibutton=wdg.Button(
    description='Get Data',
    disabled=False,
    button_style='Info',
    tooltip="This will get the latest data for you",
    icon='download'
)

apibutton.on_click(api_button_callback)

display(apibutton)

Button(button_style='info', description='Get Data', icon='download', style=ButtonStyle(), tooltip='This will g…

**Author and Copyright Notice** *This dashboard was made by [Naomi Christie](https://github.com/nchristie) Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*